In [1]:
import gc
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [2]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [3]:
data.loc[data['label'] == -1, 'label'] = 1
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0

In [4]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
data_test = pd.read_csv(csv_path)

In [5]:
data.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

In [6]:
data_train = data[data['date'] <= 20171031]
data_val = data[data['date'] > 20171031]

print(data.shape)
print(data_train.shape)
print(data_val.shape)

X = data.loc[:,'f1':].as_matrix()
y = data.loc[:,'label'].as_matrix()

X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(994731, 300)
(911606, 300)
(83125, 300)
(994731, 297)
(994731,)
(911606, 297)
(911606,)
(83125, 297)
(83125,)
(491668, 297)


In [7]:
del data_train, data_val
gc.collect()

56

In [8]:
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,1,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,1,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [18]:
data_test

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,1,20180120,0,0,1,1,100809.0,1,3,2,0,1,0,1,1,2,2,1,2,2,1,1.0,15.0,0.0,92.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,4.0,1.0,1.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,4.0,6.0,24.0,60.0,60.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,10.0,11.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,12.0,13.0,2.0,1.0,2.0,5.0,7.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,7.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,71.0,171.0,301.0,337.0,385.0,31.0,75.0,122.0,130.0,130.0,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,0,20180105,1,1,0,0,100808.0,1,5,2,2,1,1,1,2,0,1,0,0,0,1,0.0,30.0,1.0,173.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,12.0,13.0,12.0,13.0,13.0,13.0,12.0,2.0,1.0,2.0,1.0,13.0,12.0,12.0,12.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,13.0,13.0,12.0,12.0,13.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,77.0,78.0,78.0,77.0,78.0,38.0,39.0,39.0,39.0,39.0,39.0,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,0,20180109,0,0,0,0,100803.0,0,6,

In [23]:
clf = lgb.LGBMClassifier(   
                            boosting_type='gbdt',
                            num_leaves=50,
                            n_estimators=300,
                            objective='binary',
                            learning_rate=0.05,
                            feature_fraction=0.9,
                            bagging_fraction=0.8,
                            random_state=2018,
                            bagging_freq=5,
                            verbose=0,
                            n_jobs=-1     
                        )
clf.fit(X_train, y_train, eval_set=[(X_val, y_val), (X_train, y_train)], eval_metric={'l2', 'auc'}, early_stopping_rounds=50)

[1]	valid_0's auc: 0.946578	valid_0's l2: 0.226692	valid_1's auc: 0.938887	valid_1's l2: 0.226623
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.952182	valid_0's l2: 0.205702	valid_1's auc: 0.94604	valid_1's l2: 0.205553
[3]	valid_0's auc: 0.960556	valid_0's l2: 0.186747	valid_1's auc: 0.953339	valid_1's l2: 0.186537
[4]	valid_0's auc: 0.961724	valid_0's l2: 0.169636	valid_1's auc: 0.954575	valid_1's l2: 0.169388
[5]	valid_0's auc: 0.964941	valid_0's l2: 0.154252	valid_1's auc: 0.958373	valid_1's l2: 0.153944
[6]	valid_0's auc: 0.96577	valid_0's l2: 0.140346	valid_1's auc: 0.960785	valid_1's l2: 0.139993
[7]	valid_0's auc: 0.965858	valid_0's l2: 0.127821	valid_1's auc: 0.961034	valid_1's l2: 0.127424
[8]	valid_0's auc: 0.965791	valid_0's l2: 0.116499	valid_1's auc: 0.96107	valid_1's l2: 0.116062
[9]	valid_0's auc: 0.965807	valid_0's l2: 0.106272	valid_1's auc: 0.960909	valid_1's l2: 0.105813
[10]	valid_0's auc: 0.966002	valid_0's l2: 0.0970143	valid_

[82]	valid_0's auc: 0.976204	valid_0's l2: 0.00942867	valid_1's auc: 0.981514	valid_1's l2: 0.00828112
[83]	valid_0's auc: 0.976254	valid_0's l2: 0.00941646	valid_1's auc: 0.981586	valid_1's l2: 0.0082653
[84]	valid_0's auc: 0.976248	valid_0's l2: 0.00940767	valid_1's auc: 0.981664	valid_1's l2: 0.00825198
[85]	valid_0's auc: 0.976272	valid_0's l2: 0.00939414	valid_1's auc: 0.981772	valid_1's l2: 0.00823476
[86]	valid_0's auc: 0.976334	valid_0's l2: 0.00938582	valid_1's auc: 0.981978	valid_1's l2: 0.00821986
[87]	valid_0's auc: 0.976475	valid_0's l2: 0.0093737	valid_1's auc: 0.982406	valid_1's l2: 0.00820274
[88]	valid_0's auc: 0.976774	valid_0's l2: 0.00936638	valid_1's auc: 0.982535	valid_1's l2: 0.0081888
[89]	valid_0's auc: 0.976835	valid_0's l2: 0.00936031	valid_1's auc: 0.982789	valid_1's l2: 0.00817639
[90]	valid_0's auc: 0.977047	valid_0's l2: 0.0093528	valid_1's auc: 0.982907	valid_1's l2: 0.00816341
[91]	valid_0's auc: 0.977088	valid_0's l2: 0.0093486	valid_1's auc: 0.983151	

[162]	valid_0's auc: 0.980007	valid_0's l2: 0.00921486	valid_1's auc: 0.989606	valid_1's l2: 0.00750334
[163]	valid_0's auc: 0.980038	valid_0's l2: 0.0092145	valid_1's auc: 0.989659	valid_1's l2: 0.00749414
[164]	valid_0's auc: 0.980071	valid_0's l2: 0.00921269	valid_1's auc: 0.989722	valid_1's l2: 0.00748449
[165]	valid_0's auc: 0.980064	valid_0's l2: 0.00921562	valid_1's auc: 0.989772	valid_1's l2: 0.00747876
[166]	valid_0's auc: 0.980093	valid_0's l2: 0.009214	valid_1's auc: 0.989807	valid_1's l2: 0.00746737
[167]	valid_0's auc: 0.980115	valid_0's l2: 0.00920727	valid_1's auc: 0.989849	valid_1's l2: 0.00745882
[168]	valid_0's auc: 0.980167	valid_0's l2: 0.00920538	valid_1's auc: 0.989878	valid_1's l2: 0.00744885
[169]	valid_0's auc: 0.980192	valid_0's l2: 0.009205	valid_1's auc: 0.989918	valid_1's l2: 0.00744057
[170]	valid_0's auc: 0.980228	valid_0's l2: 0.009203	valid_1's auc: 0.989991	valid_1's l2: 0.00743139
[171]	valid_0's auc: 0.980245	valid_0's l2: 0.00920305	valid_1's auc: 0

[242]	valid_0's auc: 0.980285	valid_0's l2: 0.00940408	valid_1's auc: 0.992329	valid_1's l2: 0.00694269
[243]	valid_0's auc: 0.980294	valid_0's l2: 0.00940354	valid_1's auc: 0.992359	valid_1's l2: 0.0069315
Early stopping, best iteration is:
[193]	valid_0's auc: 0.980595	valid_0's l2: 0.00919556	valid_1's auc: 0.990947	valid_1's l2: 0.00725605


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=50, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=0)

In [24]:
y_pred = clf.predict_proba(X_test)[:,0]

In [12]:
count = 0
white_samples = []
for i in range(len(y_pred)):
    if y_pred[i] < 0.0009:
        count += 1
        white_samples.append(i)
        y_pred[i] = 1
print(count)

132235


In [13]:
y_pred

array([1.        , 0.03923202, 0.00111884, ..., 1.        , 1.        ,
       1.        ])

In [14]:
y_pred_int = np.array([int(x) for x in y_pred])
y_pred_int

array([1, 0, 0, ..., 1, 1, 1])

In [15]:
data_test.iloc[white_samples]

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,20180120,0,0,1,1,100809.0,1,3,2,0,1,0,1,1,2,2,1,2,2,1,1.0,15.0,0.0,92.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,4.0,1.0,1.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,4.0,6.0,24.0,60.0,60.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,10.0,11.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,12.0,13.0,2.0,1.0,2.0,5.0,7.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,7.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,71.0,171.0,301.0,337.0,385.0,31.0,75.0,122.0,130.0,130.0,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
7,b6cba0173d6a894f443578b5d82c00444dfe24c5e1705d...,20180115,0,0,0,0,100809.0,1,2,1,0,2,1,2,1,1,2,1,2,2,2,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,6.0,9.0,9.0,2.0,2.0,5.0,7.0,7.0,2.0,2.0,9.0,2.0,2.0,1.0,3.0,6.0,24.0,31.0,25.0,31.0,1.0,2.0,3.0,3.0,29.0,37.0,6.0,873.6,1008.6,116.0,206.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,7.0,10.0,10.0,4.0,8.0,26.0,33.0,33.0,4.0,9.0,26.0,33.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,8.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,4.0,3.0,9.0,26.0,33.0,32.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,7.0,8.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,5.0,7.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
10,85426887c8a4f4fd881ae1e79654940e35c68bfd681a5e...,20180122,0,0,1,0,100809.0,0,2,1,0,0,0,2,2,2,1,2,2,2,1,0

In [16]:
data_test.insert(1, 'label', value=y_pred_int)

In [19]:
data = pd.concat([data, data_test.iloc[white_samples]], axis=0)
data.shape

(1126966, 300)

In [20]:
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,1,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,1,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [21]:
data_train = data[(data['date'] <= 20171031) | (data['date'] > 20171231)]
data_val = data[(data['date'] > 20171031) & (data['date'] <= 20171231)]

print(data.shape)
print(data_train.shape)
print(data_val.shape)

(1126966, 300)
(1043841, 300)
(83125, 300)


In [22]:
X = data.loc[:,'f1':].as_matrix()
y = data.loc[:,'label'].as_matrix()

X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(1126966, 297)
(1126966,)
(1043841, 297)
(1043841,)
(83125, 297)
(83125,)
(491668, 297)


In [ ]:
# clf2 = GradientBoostingClassifier(
#                                     #n_estimators=300,
#                                     #subsample=0.8,
#                                     #max_leaf_nodes=50,
#                                     #max_features=0.9,
#                                     #learning_rate=0.05,
#                                     random_state=2018,
#                                     verbose=2
#                                 )
# clf2.fit(X, y)

In [ ]:
#y_pred2 = clf2.predict_proba(X_test)[:,0]

In [ ]:
#y_pred = y_pred1 * 0.65 + y_pred2 * 0.35

In [26]:
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("result31.csv", index=False)

In [18]:
data_test.drop('label', axis=1, inplace=True)

In [57]:
count = 0
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        count += 1
print(count)

2711
